# Week 4: Handling Complex Images - Happy or Sad Dataset

In this assignment you will be using the happy or sad dataset, which contains 80 images of emoji-like faces, 40 happy and 40 sad.

Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

Begin by taking a look at some images of the dataset:

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

happy_dir="./data/happy"
sad_dir="./data/sad"

plt.imshow(load_img(f"{os.path.join(happy_dir,os.listdir(happy_dir)[0])}"))
plt.show()

plt.imshow(load_img(f"{os.path.join(sad_dir,os.listdir(sad_dir)[0])}"))
plt.show()



It is cool to be able to see examples of the images to better understand the problem-space you are dealing with. 

However there is still some relevant information that is missing such as the resolution of the image (although matplotlib renders the images in a grid providing a good idea of these values) and the maximum pixel value (this is important for normalizing these values). For this you can use Keras as shown in the next cell:

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array

sample_img=load_img(os.path.join(happy_dir,os.listdir(happy_dir)[0]))
sample_array=img_to_array(sample_img)

print(f"image has shape{sample_array.shape}")
print(f"max pixel value : {np.max(sample_array)}")


Looks like the images have a resolution of 150x150. **This is very important because this will be the input size of the first layer in your network.** 

**The last dimension refers to each one of the 3 RGB channels that are used to represent colored images.**

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epochs,logs=[]):
        if logs.get('accuracy')>0.999 and logs.get('accuracy') is not None:
            print("reached 99.9% accuracy")
            self.model.stop_training=True


A quick note on callbacks: 

So far you have used only the `on_epoch_end` callback but there are many more. For example you might want to check out the [EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping) callback, which allows you to save the best weights for your model.

Keras provides great support for preprocessing image data. A lot can be accomplished by using the `ImageDataGenerator` class. Be sure to check out the [docs](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) if you get stuck in the next exercise. In particular you might want to pay attention to the `rescale` argument when instantiating the `ImageDataGenerator` and to the [`flow_from_directory`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory) method.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def image_generator():

    train_data=ImageDataGenerator(rescale=1/255.0)

    train_generator=train_data.flow_from_directory(directory='./data/',tagret_size=(150,150),batch_size=10,class_mode='binary')
    return train_generator

In [ ]:
generated=image_generator()

**Expected Output:**
```
Found 80 images belonging to 2 classes.
```

In [ ]:
def train_happy_sad_model(train_gen):
    callbacks=myCallback()


    model=tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        tf.keras.layers.Flatten(),
        
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),loss='binary_crossentropy',metrics=['accuracy'])
    history=model.fit(train_gen,epochs=20,callbacks=[callbacks])
    return history


In [ ]:
hist = train_happy_sad_model(generated)

If you see the message that was defined in the callback printed out after less than 15 epochs it means your callback worked as expected and training was successful. You can also double check by running the following cell:

In [ ]:
print(f"Your model reached the desired accuracy after {len(hist.epoch)} epochs")

**Congratulations on finishing the last assignment of this course!**

You have successfully implemented a CNN to assist you in the classification task for complex images. Nice job!

**Keep it up!**